<a href="https://colab.research.google.com/github/jmcava/projeto_extracao_facebook_oficial/blob/main/projeto_IL_final_extracao_e_mineracao_dos_dados_facebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação das bibliotecas**

In [ ]:
!pip install facebook-sdk

In [ ]:
import facebook
import requests
import pandas as pd
import numpy as np


In [ ]:
!pip install plotly --upgrade

     |████████████████████████████████| 15.2 MB 5.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
import plotly.express as px

**Obtenção token temporário**

In [ ]:
app_secret = '3584b509a6652ef4a49ae7eb68a625f2'
# 1 hora
user_short_token = 'EAAZAUTnCFGMEBAKmgaK1AMoO5LeVbZAtGW8A5ErogtzZAQ38uONPTfPVF2SRWuXGuOeLoz31Qpfg7VQRdatLezkPdX9ucB7YKfOZBkcacU31ykB2EpZAuxzjOtoYKTaPqPHZC61FKtRZCfKhaxBZBi8dT9uGt1kRZA9g9lTk5H2DBZC68Qi7qFOZBw146h87P6i0uUoZAAQZApRZCZCSo8xo9dQGJsafLpHy2EyMhWfpOP3tLbU8uMrDc1x8veieZBqzdyZBx2XwZD'

In [ ]:
url = 'https://graph.facebook.com/oauth/access_token'

In [ ]:
parameters = {'grant_type': 'fb_exchange_token',
            'client_id': app_id,
            'client_secret': app_secret,
            'fb_exchange_token': user_short_token}

NameError: ignored

In [ ]:
response = requests.get(url, params=parameters)

In [ ]:
response

In [ ]:
response_json = response.json()
response_json

In [ ]:
# 2 meses
user_long_token = response_json['access_token']
user_long_token

**Obtenção de token permanente**

In [ ]:
graph = facebook.GraphAPI(access_token=user_long_token, version='3.1')

In [ ]:


dados_pagina = graph.get_object('/me/accounts')
dados_pagina

In [ ]:
permanente_page_token = dados_pagina['data'][0]['access_token']
permanente_page_token

In [ ]:
page_id = dados_pagina['data'][0]['id']
page_id

**Informações básicas da página**

. Documentação: https://developers.facebook.com/docs/graph-api




In [ ]:
graph = facebook.GraphAPI(access_token=permanente_page_token, version = '3.1')

In [ ]:
graph.get_object(id=page_id, fields='category')

In [ ]:
graph.get_object(id=page_id, fields='name')

In [ ]:
graph.get_object(id=page_id)

In [ ]:
graph.get_object(id=page_id, fields='about,website')

In [ ]:
graph.get_object(id=page_id, fields='fan_count')

In [ ]:
graph.get_object(id=page_id, fields='birthday')

In [ ]:
graph.get_object(id=page_id, fields='engagement')

In [ ]:
graph.get_object(id=page_id, fields='followers_count')

**Informações sobre os fãs da página**

Fãs por data

In [ ]:
graph.get_connections(id = page_id, connection_name='insights', metric='page_fans')

In [ ]:
fas = graph.get_connections(id=page_id, connection_name='insights', metric='page_fans',
                            since = '2022-02-02', until = '2022-05-06')

In [ ]:
type(fas), type(fas['data'])

In [ ]:
fas

In [ ]:
fas['data'][0]

In [ ]:
fas ['data'][0]['values']

In [ ]:
dataframe_facebook = pd.DataFrame(fas['data'][0]['values'])#criarção dataFrame
dataframe_facebook.rename(columns={'value':'numero fas','end_time':'data'},inplace=True)
dataframe_facebook['data'] = pd.to_datetime(dataframe_facebook['data'])
dataframe_facebook['data'] = dataframe_facebook['data'].dt.strftime('%Y-%m-%d')
dataframe_facebook

In [ ]:
grafico = px.line(dataframe_facebook, x = 'data', y = 'numero fas', title = 'Numero de fãs por dia')
grafico.show()


Fãs por idioma

In [ ]:
fas_idioma = graph.get_connections(id=page_id, connection_name='insights', metric = 'page_fans_locale',
                                   since = '2022-02-02', until = '2022-05-04')
fas_idioma

In [ ]:
fas_idioma['data'][0]['values'][0]['value']#monta o dataFrame dos idiomas

In [ ]:
fas_idioma_df = pd.DataFrame.from_dict(fas_idioma['data'][0]['values'][0]['value'], orient='index')
fas_idioma_df.reset_index(inplace=True)
fas_idioma_df.rename(columns={'index': 'idioma', 0: 'quantidade'}, inplace=True)
fas_idioma_df


In [ ]:
grafico = px.bar(fas_idioma_df, x='idioma', y='quantidade', title='Fãs por idioma')
grafico.show()

Fãs por cidade

In [ ]:
fas_cidade = graph.get_connections(id=page_id, connection_name='insights', metric='page_fans_city',
                                   since = '2022-02-02', until='2022-05-04')
fas_cidade

In [ ]:
fas_cidade['data'][0]['values'][0]['value']

In [ ]:
fas_cidade_df = pd.DataFrame.from_dict(fas_cidade['data'][0]['values'][0]['value'], orient='index')
fas_cidade_df.reset_index(inplace=True)
fas_cidade_df.rename(columns={'index': 'cidade', 0: 'quantidade'}, inplace=True)
fas_cidade_df

In [ ]:
grafico =px.bar(fas_cidade_df, x = 'cidade', y='quantidade', title = 'Fãs por cidade')
grafico.show()

In [ ]:
grafico = px.treemap(fas_cidade_df, path = ['cidade'])
grafico

In [ ]:
fas_pais = graph.get_connections(id=page_id, connection_name='insights', metric='page_fans_country',
                                   since = '2022-02-02', until='2022-05-04')
fas_pais

In [ ]:
fas_pais['data'][0]['values'][0]['value']

In [ ]:
fas_pais_df = pd.DataFrame.from_dict(fas_pais['data'][0]['values'][0]['value'], orient='index')
fas_pais_df.reset_index(inplace=True)
fas_pais_df.rename(columns={"index": "pais", 0: "quantidade"}, inplace=True)
fas_pais_df

In [ ]:
grafico = px.bar(fas_pais_df, x="pais", y="quantidade", title = 'Fãs por país')
grafico.show()

In [ ]:
grafico = px.treemap(fas_pais_df, path = ['pais'])
grafico

Fãs por idade e gênero

In [ ]:
fas_idade_genero = graph.get_connections(id=page_id, connection_name='insights', metric='page_fans_gender_age',
                                   since = '2022-02-02', until='2022-05-04')
fas_idade_genero

In [ ]:
fas_idade_genero_df = pd.DataFrame.from_dict(fas_idade_genero['data'][0]['values'][0]['value'], orient='index')
fas_idade_genero_df.reset_index(inplace=True)
fas_idade_genero_df.rename(columns={"index": "idade genero", 0: "quantidade"}, inplace=True)
fas_idade_genero_df

In [ ]:
grafico = px.bar(fas_idade_genero_df, x="idade genero", y="quantidade", title = 'Fãs por idade e gênero')
grafico.show()

Curtidas e descurtidas por período

In [ ]:
curtidas = graph.get_connections(id=page_id, connection_name='insights', metric='page_fan_adds',
                                 since = '2022-02-02', until='2022-05-04')
curtidas


In [ ]:
curtidas['data'][0]['values']

In [ ]:
numero_curtidas = []
for i in curtidas['data'][0]['values']:
  #print(i['value'])
  numero_curtidas.append(i['value'])
numero_curtidas = np.array(numero_curtidas)#conversão de uma lista para um array

In [ ]:
numero_curtidas

In [ ]:
descurtidas = graph.get_connections(id=page_id, connection_name='insights', metric='page_fan_removes',
                                    since = '2022-02-02', until = '2022-05-04')
descurtidas

In [ ]:
numero_descurtidas = []
for i in descurtidas['data'][0]['values']:
  numero_descurtidas.append(i['value'])
numero_descurtidas = np.array(numero_descurtidas)

In [ ]:
numero_descurtidas

In [ ]:
len(dataframe_facebook), len(numero_curtidas), len(numero_descurtidas)

In [ ]:
dataframe_facebook['numero curtidas'] = numero_curtidas
dataframe_facebook['numero descurtidas'] = numero_descurtidas
dataframe_facebook

In [ ]:
dataframe_facebook.describe()

In [ ]:
dataframe_facebook[dataframe_facebook['numero curtidas'] == 18]

In [ ]:
dataframe_facebook[dataframe_facebook['numero descurtidas'] == 20]

In [ ]:
grafico = px.line(title = 'Curtidas x Descurtidas')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero curtidas'], name = 'Curtidas')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero descurtidas'], name = 'Descurtidas')
grafico.show()

In [ ]:
dataframe_facebook['numero curtidas'].sum(), dataframe_facebook['numero descurtidas'].sum()

In [ ]:
166 + 27, (27 / 193) * 100 # a porcentagem em relação a 166 curtidas equivalente a 13,99% onde o resultado são 27 descurtidas

In [ ]:
descurtidas_tipo = graph.get_connections(id=page_id, connection_name='insights', metric='page_fans_by_unlike_source_unique',
                                             since = '2022-02-02', until = '2022-05-04')
descurtidas_tipo

In [ ]:
motivos = {}
for i in descurtidas_tipo['data'][0]['values']:
  print(i)

In [ ]:
motivos = {}
for i in descurtidas_tipo['data'][0]['values']:
  #print(i)
  if i ['value']:
    #print(i)
    print('---------------')
    print(i['value'])
    for k, v in i['value'].items():
      print(k,v)

In [ ]:
motivos = {}
for i in descurtidas_tipo['data'][0]['values']:
  if i['value']:
    for k in i['value'].items():
      if k in motivos:
        motivos[k] += v
      else:
        motivos[k] = v

In [ ]:
motivos

In [ ]:
motivos_df = pd.DataFrame.from_dict(motivos, orient='index')
motivos_df.reset_index(inplace=True)
motivos_df.rename(columns={'index': 'motivo', 0: 'quantidade'}, inplace=True)
motivos_df


In [ ]:
grafico = px.bar(motivos_df, x = 'motivo', y = 'quantidade', title = 'Motivos das descurtidas na página')
grafico.show()

In [ ]:
166 + 8, (8 / 174) * 100 # ou seja 4,59% dos posts obtiveram negativas em diversas

VISUALIZAÇÃO DE ABAS

In [ ]:
visualizacao_abas = graph.get_connections(id=page_id, connection_name='insights', metric = 'page_tab_views_login_top_unique',
                                          since = '2022-02-02', until = '2022-05-04')
visualizacao_abas

# Nova seção

# Nova seção

In [ ]:
fonte_visualizacao = {}
for i in visualizacao_abas['data'][0]['values']:
  print(i)
  if i['value']:
    for k, v in i['value'].items():
      print(k,v)

In [ ]:
fonte_visualizacao = {}
for i in visualizacao_abas['data'][0]['values']:
  #print(i)
  if i['value']:
    for k, v in i['value'].items():
      #print(k,v)
      if k in fonte_visualizacao:
        fonte_visualizacao[k] +=v
      else:
        fonte_visualizacao[k] = v

In [ ]:
fonte_visualizacao

In [ ]:
fonte_visualizacao_df = pd.DataFrame.from_dict(fonte_visualizacao, orient='index')
fonte_visualizacao_df.reset_index(inplace=True)
fonte_visualizacao_df.rename(columns={"index": "fonte", 0: "quantidade"}, inplace=True)
fonte_visualizacao_df

In [ ]:
grafico = px.bar(fonte_visualizacao_df, x='fonte', y="quantidade", title = "Visualização por abas")
grafico.show()

In [ ]:
cliques_contato_acao = graph.get_connections(id=page_id, connection_name='insights', metric = 'page_total_actions',
                                             since = '2022-02-02', until = '2022-05-04')
cliques_contato_acao


In [ ]:
numero_cliques_contato_acao = []
for i in cliques_contato_acao['data'][0]['values']:
  print(i)

In [ ]:
numero_cliques_contato_acao = []
for i in cliques_contato_acao['data'][0]['values']:
  #print(i)
  numero_cliques_contato_acao.append(i['value'])
numero_cliques_contato_acao = np.array(numero_cliques_contato_acao)

In [ ]:
numero_cliques_contato_acao

In [ ]:
len(numero_cliques_contato_acao)

In [ ]:
dataframe_facebook['numero cliques contato acao'] = numero_cliques_contato_acao
dataframe_facebook

In [ ]:
dataframe_facebook['numero cliques contato acao'].describe()

In [ ]:
grafico = px.line(dataframe_facebook, x="data", y="numero cliques contato acao", title = 'Número de cliques em contato')
grafico.show()

In [ ]:
engajamento = graph.get_connections(id=page_id, connection_name='insights', metric='page_engaged_users',
                                        since = '2022-02-02', until = '2022-05-04')
engajamento

In [ ]:
engajamento_cliques = []
for i in engajamento['data'][0]['values']:
  engajamento_cliques.append(i['value'])
emgajamento_cliques = np.array(engajamento_cliques)


In [ ]:
engajamento_cliques

In [ ]:
len(engajamento_cliques)

In [ ]:
dataframe_facebook['numero cliques engajamento'] = engajamento_cliques
dataframe_facebook

In [ ]:
grafico = px.line(dataframe_facebook, x="data", y="numero cliques engajamento", title = 'Engajamento - numero de cliques')
grafico.show()

In [ ]:
engajamento_tipo = graph.get_connections(id=page_id, connection_name='insights', metric='page_consumptions_by_consumption_type',
                                         since = '2022-02-02', until = '2022-05-04')
engajamento_tipo

In [ ]:
fonte_engajamento = {}
for i in engajamento_tipo['data'][0]['values']:
  if i['value']:
    for k, v in i['value'].items():
      if k in fonte_engajamento:
        fonte_engajamento[k] += v
      else:
        fonte_engajamento[k] = v

In [ ]:
fonte_engajamento

In [ ]:
fonte_engajamento_df = pd.DataFrame.from_dict(fonte_engajamento, orient='index')
fonte_engajamento_df.reset_index(inplace=True)
fonte_engajamento_df.rename(columns={"index": "fonte", 0: "quantidade"}, inplace=True)
fonte_engajamento_df

In [ ]:
grafico = px.bar(fonte_engajamento_df, x='fonte', y="quantidade", title = 'Fonte do engajamento')
grafico.show()

In [ ]:
graph.get_connections(id=page_id, connection_name='insights', metric='page_places_checkin_total')

In [ ]:
acoes_positivas = graph.get_connections(id=page_id, connection_name='insights', metric='page_positive_feedback_by_type',
                                        since = '2022-02-02', until = '2022-05-04')

In [ ]:
acoes_positivas

In [ ]:
fonte_positivas = {}
for i in acoes_positivas['data'][0]['values']:
  if i['value']:
    for k, v in i['value'].items():
      if k in fonte_positivas:
        fonte_positivas[k] += v
      else:
        fonte_positivas[k] = v


In [ ]:
fonte_positivas

In [ ]:
fonte_positivas_df = pd.DataFrame.from_dict(fonte_positivas, orient='index')
fonte_positivas_df.reset_index(inplace=True)
fonte_positivas_df.rename(columns={"index": "fonte", 0: "quantidade"}, inplace=True)
fonte_positivas_df

In [ ]:
grafico = px.bar(fonte_positivas_df, x='fonte', y="quantidade", title = 'Fonte do positivas')
grafico.show()

In [ ]:
acoes_negativas = graph.get_connections(id=page_id, connection_name='insights', metric='page_negative_feedback_by_type',
                                        since = '2022-02-02', until = '2022-05-04')
acoes_negativas

In [ ]:
fonte_negativas = {}
for i in acoes_negativas['data'][0]['values']:
  if i['value']:
    for k, v in i['value'].items():
      if k in fonte_negativas:
        fonte_negativas[k] += v
      else:
        fonte_negativas[k] = v

In [ ]:
fonte_negativas

In [ ]:
hora_dia = graph.get_connections(id=page_id, connection_name='insights', metric='page_fans_online',
                                        since = '2022-02-02', until = '2022-05-04')
hora_dia

In [ ]:
visualizacao_hora = {}
for i in hora_dia['data'][0]['values']:
  if i['value']:
    for k, v in i['value'].items():
      if k in visualizacao_hora:
        visualizacao_hora[k] += v
      else:
        visualizacao_hora[k] = v

In [ ]:
visualizacao_hora

In [ ]:
hora_dia_df = pd.DataFrame.from_dict(visualizacao_hora, orient='index')
hora_dia_df.reset_index(inplace=True)
hora_dia_df.rename(columns={"index": "hora", 0: "quantidade"}, inplace=True)
hora_dia_df

In [ ]:
grafico = px.bar(hora_dia_df, x='hora', y="quantidade", title = 'Visualizações por hora do dia')
grafico.show()

In [ ]:
curtidas_pagas_nao_pagas = graph.get_connections(id=page_id, connection_name='insights',
                                                 metric='page_fan_adds_by_paid_non_paid_unique',
                                                 since = '2022-02-02', until = '2022-05-04')
curtidas_pagas_nao_pagas

In [ ]:
curtidas_pagas = []
curtidas_nao_pagas = []
for i in curtidas_pagas_nao_pagas['data'][0]['values']:
  print(i, i['value']['paid'], i['value']['unpaid'])
  curtidas_pagas.append(i['value']['paid'])
  curtidas_nao_pagas.append(i['value']['unpaid'])
curtidas_pagas = np.array(curtidas_pagas)
curtidas_nao_pagas = np.array(curtidas_nao_pagas)

In [ ]:
curtidas_pagas

In [ ]:
curtidas_nao_pagas

In [ ]:
dataframe_facebook['curtidas pagas'] = curtidas_pagas
dataframe_facebook['curtidas nao pagas'] = curtidas_nao_pagas
dataframe_facebook

In [ ]:
grafico = px.line(title = 'Curtidas pagas x curtidas não pagas')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['curtidas pagas'], name = 'Curtidas pagas')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['curtidas nao pagas'], name = 'Curtidas não paga')
grafico.show()

In [ ]:
distribuicao_paga = graph.get_connections(id=page_id, connection_name='insights', metric='page_impressions_paid',
                                                 since = '2022-02-02', until = '2022-05-04')
distribuicao_paga

In [ ]:
numero_distribuicao_paga = []
for i in distribuicao_paga['data'][0]['values']:
  numero_distribuicao_paga.append(i['value'])
numero_distribuicao_paga = np.array(numero_distribuicao_paga)

In [ ]:
numero_distribuicao_paga

In [ ]:
distribuicao_organica = graph.get_connections(id=page_id, connection_name='insights',
                                                 metric='page_impressions_organic',
                                                 since = '2022-02-02', until = '2022-05-04')
distribuicao_organica

In [ ]:
numero_distribuicao_organica = []
for i in distribuicao_organica['data'][0]['values']:
  numero_distribuicao_organica.append(i['value'])
numero_distribuicao_organica = np.array(numero_distribuicao_organica)

In [ ]:
numero_distribuicao_organica

In [ ]:
distribuicao_viral = graph.get_connections(id=page_id, connection_name='insights',
                                                 metric='page_impressions_viral',
                                                 since = '2022-02-02', until = '2022-05-04')
distribuicao_viral

In [ ]:
numero_distribuicao_viral = []
for i in distribuicao_viral['data'][0]['values']:
  numero_distribuicao_viral.append(i['value'])
numero_distribuicao_viral = np.array(numero_distribuicao_viral)

In [ ]:
numero_distribuicao_viral

In [ ]:
dataframe_facebook['numero distribuicao paga'] = numero_distribuicao_paga
dataframe_facebook['numero distribuicao organica'] = numero_distribuicao_organica
dataframe_facebook['numero distribuicao viral'] = numero_distribuicao_viral
dataframe_facebook

In [ ]:
grafico = px.line(title = 'Distribuição paga x orgânica x viral')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero distribuicao paga'], name = 'Paga')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero distribuicao organica'], name = 'Organica')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero distribuicao viral'], name = 'Viral')
grafico.show()

In [ ]:
dataframe_facebook.columns

In [ ]:
dataframe_distribuicao = dataframe_facebook.iloc[:, [1,8,9,10]]
dataframe_distribuicao

In [ ]:
dataframe_distribuicao_normalizado = dataframe_distribuicao.copy()
for i in dataframe_distribuicao_normalizado.columns[1:]:
  #print(i)
  dataframe_distribuicao_normalizado[i] = dataframe_distribuicao_normalizado[i] / dataframe_distribuicao_normalizado[i][0]

In [ ]:
dataframe_distribuicao_normalizado

In [ ]:
grafico = px.line(title = 'Distribuição paga x orgânica x viral - normalizado')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero distribuicao paga'], name = 'Paga')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero distribuicao organica'], name = 'Organica')
grafico.add_scatter(x = dataframe_facebook['data'], y = dataframe_facebook['numero distribuicao viral'], name = 'Viral')
grafico.show()

In [ ]:
!pip install pmdarima

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima

In [ ]:
dataframe_facebook

In [ ]:
dataframe_facebook.to_csv('dataframe_facebook.csv')

In [ ]:
import datetime
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')
dataframe_facebook = pd.read_csv('/content/dataframe_facebook.csv', parse_dates=['data'], index_col = 'data', date_parser = dateparse)
dataframe_facebook

In [ ]:
ts_fas = dataframe_facebook.iloc[:, [1]]
ts_fas

In [ ]:
decomposicao = seasonal_decompose(ts_fas)
tendencia = decomposicao.trend
sazonal = decomposicao.seasonal
residuo = decomposicao.resid

In [ ]:
grafico = px.line(ts_fas, y = 'numero fas', title = 'Número de fãs por dia')
grafico.show()

In [ ]:
grafico = px.line(tendencia, title = 'Tendência')
grafico.show()

In [ ]:
grafico = px.line(sazonal, title = 'Sazonalidade')
grafico.show()

In [ ]:
grafico = px.line(residuo, title = 'Residuos')
grafico.show()

In [ ]:
arima = auto_arima(ts_fas)

In [ ]:
arima.order

In [ ]:
previsoes = pd.DataFrame(arima.predict(n_periods=30), columns=['previsao'])
previsoes['previsao'] = np.round(previsoes['previsao']).astype(int)
previsoes

In [ ]:
dias = pd.date_range(start = '2022-04-01', end = '2022-04-30')
dias

In [ ]:
previsoes['data'] = dias
previsoes.set_index('data', inplace=True)
previsoes

In [ ]:
fas_teste = graph.get_connections(id=page_id, connection_name='insights',
                                  metric = 'page_fans', since = '2022-04-01' )
fas_teste

In [ ]:
fas_teste['data'][0]['values']

In [ ]:
fas_teste_df = pd.DataFrame(fas_teste['data'][0]['values'])
fas_teste_df.rename(columns={'value': 'numero fas', 'end_time': 'data'}, inplace=True)
fas_teste_df['data'] = pd.to_datetime(fas_teste_df['data'])
fas_teste_df['data'] = fas_teste_df['data'].dt.strftime('%Y-%m-%d')
fas_teste_df.set_index('data', inplace=True)
fas_teste_df

In [ ]:
previsoes['2022-04-01':'2022-04-30']

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(fas_teste_df['numero fas'], previsoes['2022-04-01':'2022-04-30']['previsao'])

In [ ]:
previsoes['2022-04-01':'2022-04-30']['previsao'] - fas_teste_df['numero fas']

In [ ]:
sum(abs(previsoes['2022-04-26':'2022-04-30']['previsao'] - fas_teste_df['numero fas'])) / len(fas_teste_df)